# Custom Prompt Template
---

Vamos a suponer que queremos que nuestro LLM (Larga Language Model) genere descripciones para una funcion al cual proprocionamos su nombre.

Para llevar a cabo esta tarea, necesitamos crear un "prompt" y su respesctivo "template", donde proporcionemos el nombre de la funcion


### ¿Por qué se necesitan plantillas de "prompts" personalizados?
----

LangChain proporciona un conjunto "prompts tempaltes" que se pueden usar para una variedad de tareas. Sin embargo, puede haber casos en los que las plantillas de "prompts" predeterminadas no satisfagan sus necesidades. Por ejemplo, es posible que desee crear una plantilla de mensajes con instrucciones dinámicas específicas para su modelo de lenguaje. En tales casos, puede crear una plantilla personalizada.

### Creando una plantilla de prompt personalizada (custom prompt template)
---

Escencialmente hay dos tipos distintos de plantilla disponibles:

- StringPromptTemplate
- ChatPromptTemplates

**StringPromptTemplate** proporciona un mensaje simple en formato de cadena, mientras que **ChatPromptTemplates** produce un mensaje más estructurado para usar con una API de chat.

En esta guia, crearemos un prompt personalizado (custom prompt) usando **StringPromptTemplate**

In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

load_dotenv(find_dotenv())

True

In [2]:
PROMPT = """\
Given the function name and source code, generate an English language explanation of the funciont.
Function Name: {function_name}
Source Code:
===
{source_code}
===
Explanation:
"""

**Nota:** Los prompts preferentemente debemos crearlos en ingles, ya que 

In [3]:
import inspect


def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

In [7]:
class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input varaibles are correct."""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable")
        return v
    
    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code = get_source_code(kwargs["function_name"])

        # Generate the prompto to be sent to the language model
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, 
            source_code=source_code
        )

        return prompt
    
    def _prompt_type(self):
        return "function-explainer"

### Ejemplo de uso
---

Ahora que hemos creado una plantilla de mensajes personalizada, podemos usarla para generar mensajes para nuestra tarea.

In [10]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generaremos un prompt para la funcion "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)

print(prompt)

Given the function name and source code, generate an English language explanation of the funciont.
Function Name: get_source_code
Source Code:
===
def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

===
Explanation:

